# Unsupervised Learning: Clustering Lab





In [5]:
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.metrics import silhouette_score
from sklearn.base import ClusterMixin
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. (50%) Implement the k-means clustering algorithm and the HAC (Hierarchical Agglomerative Clustering) algorithm.

### 1.1.1 HAC

### Code requirements 
- HAC should support both single link and complete link options.
- HAC automatically generates all clusterings from n to 1.  To simplify the amount of output you may want to implement a mechanism to specify for which k values actual output will be generated.


---
The output should include the following:
- The number of clusters (k).
- The silhouette score of the full clustering. (You can either write and use your own silhouette_score function (extra credit) or use sklearn's)


For each cluster report include:


- The centroid id.
- The number of instances tied to that centroid. 
---

In [15]:
from scipy.spatial import distance
class HACClustering(BaseEstimator,ClusterMixin):

    def __init__(self,k=3,link_type='single',dist_type='manhattan'): ## add parameters here
        """
        Args:
            k = how many final clusters to have
            link_type = single or complete. when combining two clusters use complete link or single link
        """
        self.link_type = link_type
        self.k = k
        self.dist_type = dist_type
        
    def fit(self, X, y=None):
        """ Fit the data; In this lab this will make the K clusters :D
        Args:
            X (array-like): A 2D numpy array with the training data
            y (array-like): An optional argument. Clustering is usually unsupervised so you don't need labels
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        self.clusters = self.init_clusters(X) # 2D python list
        # self.dist_mat = self.calc_distance(X) # 2D numpy array
        self.dist_mat = distance.cdist(X, X, metric='euclidean') # https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.cdist.html
        while  len(self.clusters) > self.k:
          nearest_clus_dist = None
          clusters_to_combine = None
          
          for c1 in range(1,len(self.clusters) -2):
            for c2 in range(c1+1,len(self.clusters) -1):
              # print('self.clusters[c1]:', self.clusters[c1])
              # print('self.clusters[c2]:', self.clusters[c2])
              dists = self.dist_mat[self.clusters[c1],[c2]]
              cur_cluster_dist = None
              if self.link_type == 'single':
                # print('c1: ', c1)
                # print('c2: ', c2)
                # print('dist1: ', dist1)
                # print('dist2: ', dist2)
                cur_cluster_dist = np.min(dists)
              else:
                cur_cluster_dist = np.max(dists)
              if nearest_clus_dist == None or cur_cluster_dist < nearest_clus_dist:
                nearest_clus_dist = cur_cluster_dist
                clusters_to_combine = [c1,c2]

          c1_indx = clusters_to_combine[0]
          c2_indx = clusters_to_combine[1]
          clusters_to_add = self.clusters[c2_indx]

          self.clusters.pop(c2)
          self.clusters[c1_indx].extend(clusters_to_add)
  
        return self

    def calc_distance(self,X):
      # variable declarations
      inst = None
      manhat_dists = []
      euclid_dists = []
      # compute manhattan and euclidean distances
      for i in range(X.shape[0]):
        inst = X[i]
        manhat_dists.append(np.linalg.norm(X - inst, axis=1, ord=1))
        euclid_dists.append(np.linalg.norm(X - inst, axis=1))
      # return the appropriate distance
      if self.dist_type == 'manhattan': 
        return np.array(manhat_dists) # 2D numpy array
      return np.array(euclid_dists) # 2D numpy array

    def init_clusters(self,X):
      clusters = []
      for i in range(X.shape[0]):
        clusters.append([i])
      return clusters
    
    def print_clusters(self):
        """
            Used for grading.
            print("Num clusters: {:d}\n".format(k))
            print("Silhouette score: {:.4f}\n\n".format(silhouette_score))
            for each cluster and centroid:
                print(np.array2string(centroid,precision=4,separator=","))
                print("{:d}\n".format(size of cluster))
        """
        print(self.clusters)

In [ ]:
arr = np.arange(10)
arr = np.reshape(arr,(5,2))
print(arr)

euclid_dists = []
manhat_dists = []
for i in range(arr.shape[0]):
  inst = arr[i]
  # others = list(range(arr.shape[0]))
  # others.remove(i)
  # others = arr[others,:]
  manhat_dists.append(np.linalg.norm(arr - inst, axis=1, ord=1))
  euclid_dists.append(np.linalg.norm(arr - inst, axis=1))

euclid_dists = np.array(euclid_dists)
manhat_dists = np.array(manhat_dists)
print('euclid_dists:\n', euclid_dists)
print('manhat_dists:\n', manhat_dists)
rows, cols = np.nonzero(euclid_dists)
print(rows)
print(cols)
print(np.argmin(euclid_dists[rows,cols]))

[[0 1]
 [2 3]
 [4 5]
 [6 7]
 [8 9]]
euclid_dists:
 [[ 0.          2.82842712  5.65685425  8.48528137 11.3137085 ]
 [ 2.82842712  0.          2.82842712  5.65685425  8.48528137]
 [ 5.65685425  2.82842712  0.          2.82842712  5.65685425]
 [ 8.48528137  5.65685425  2.82842712  0.          2.82842712]
 [11.3137085   8.48528137  5.65685425  2.82842712  0.        ]]
manhat_dists:
 [[ 0.  4.  8. 12. 16.]
 [ 4.  0.  4.  8. 12.]
 [ 8.  4.  0.  4.  8.]
 [12.  8.  4.  0.  4.]
 [16. 12.  8.  4.  0.]]
[0 0 0 0 1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4]
[1 2 3 4 0 2 3 4 0 1 3 4 0 1 2 4 0 1 2 3]
0


In [ ]:
array = [[0],[1],[2],[3],[4],[5]]
array.pop(0)
add = [0]
array[0].extend(add)
print(array)


[[1, 0], [2], [3], [4], [5]]


### 1.1.2 Debug 

Debug your model by running it on the [Debug Abalone Dataset](https://byu.instructure.com/courses/14142/files?preview=4735805)


---
The dataset was modified to be a lot smaller. The last datapoint should be on line 359 or the point 0.585,0.46,0.185,0.922,0.3635,0.213,0.285,10. The remaining points should be commented out.


- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values.


---
Solutions in files:

[Debug HAC Single (Silhouette).txt](https://byu.instructure.com/courses/14142/files?preview=4735819)

[Debug HAC Complete (Silhouette).txt](https://byu.instructure.com/courses/14142/files?preview=4735820)

In [16]:
# Debug Here
from scipy.io.arff import loadarff 
import pandas as pd
from sklearn.preprocessing import LabelEncoder
# Load data
raw_data = loadarff('/content/drive/MyDrive/School/CS_472_MachLearning/labs/lab5_clustering/data/abalone.arff')
df_data = pd.DataFrame(raw_data[0])

np_arr = df_data.to_numpy()

X = np_arr[:,:].astype(float)

hac = HACClustering(k=3,link_type='single',dist_type='euclidean')
hac.fit(X)

hac.print_clusters()


[[0], [1, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 2, 3], [199]]


### 1.1.3 Evaluation

We will evaluate your model based on its print_clusters() output using [Evaluation Seismic-bumps_train Dataset](https://byu.instructure.com/courses/14142/files?preview=4735829)

- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values.

#### 1.1.3.1 Complete Link

In [ ]:
# Load evaluation data

# Train on evaluation data using complete link

# Print clusters

#### 1.1.3.1 Single Link

In [ ]:
# Load evaluation data

# Train on evaluation data using single link

# Print clusters

### 1.2.1 K-Means

### Code requirements 
- Ability to choose k and specify k initial centroids
- Use Euclidean Distance as metric
- Ability to handle distance ties
- Include output label as a cluster feature


---
The output should include the following:
- The number of clusters (k).
- The silhouette score of the full clustering. (You can either write and use your own silhouette_score function (extra credit) or use sklearn's)


For each cluster report include:


- The centroid id.
- The number of instances tied to that centroid. 
---
You only need to handle continuous features

In [ ]:
class KMEANSClustering(BaseEstimator,ClusterMixin):

    def __init__(self,k=3,debug=False): ## add parameters here
        """
        Args:
            k = how many final clusters to have
            debug = if debug is true use the first k instances as the initial centroids otherwise choose random points as the initial centroids.
        """
        self.k = k
        self.debug = debug

    def fit(self, X, y=None):
        """ Fit the data; In this lab this will make the K clusters :D
        Args:
            X (array-like): A 2D numpy array with the training data
            y (array-like): An optional argument. Clustering is usually unsupervised so you don't need labels
        Returns:
            self: this allows this to be chained, e.g. model.fit(X,y).predict(X_test)
        """
        return self
    
    def print_clusters(self):
        """
            Used for grading.
            print("Num clusters: {:d}\n".format(k))
            print("Silhouette score: {:.4f}\n\n".format(silhouette_score))
            for each cluster and centroid:
                print(np.array2string(centroid,precision=4,separator=","))
                print("{:d}\n".format(size of cluster))
        """
        pass

### 1.2.2 Debug 

Debug your model by running it on the [Debug Abalone Dataset](https://byu.instructure.com/courses/14142/files?preview=4735805)


- Train until convergence
- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use the first k instances as the initial centroids
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values




---
Solutions in files:

[Debug K Means (Silhouette).txt](https://byu.instructure.com/courses/14142/files?preview=4735840)

In [ ]:
# Load debug data

# Train on debug data

# Print clusters

### 1.2.3 Evaluation

We will evaluate your model based on its print_clusters() output using [Evaluation Seismic-bumps_train Dataset](https://byu.instructure.com/courses/14142/files?preview=4735829)
- Train until convergence
- Make sure to include the output class (last column) as an additional input feature
- Normalize Data
- K = 5
- Use the first k instances as the initial centroids
- Use 4 decimal places and DO NOT ROUND when reporting silhouette score and centroid values

In [ ]:
# Load evaluation data

# Train on evaluation data

# Print clusters

## 2.1.1 (7.5%) Clustering the Iris Classification problem - HAC

Load the Iris Dataset [Iris Dataset](https://byu.instructure.com/courses/14142/files?preview=4421369)

- Use single-link and complete link clustering algorithms
- State whether you normalize your data or not (your choice).  
- Show your results for clusterings using k = 2-7.  
- Graph the silhouette score for each k and discuss your results (i.e. what kind of clusters are being made).
---

In [ ]:
# Iris Classification using single-link

In [ ]:
# Iris Classification using complete-link

Discuss differences between single-link and complete-link

## 2.1.2 (5%) Clustering the Iris Classification problem - HAC

Requirements:
- Repeat excercise 2.1.1 and include the output label as one of the input features.

In [ ]:
# Clustering Labels using single-link

In [ ]:
# Clustering Labels using complete-link

Discuss any differences between the results from 2.1.1 and 2.1.2.

## 2.2.1 (7.5%) Clustering the Iris Classification problem: K-Means

Load the Iris Dataset [Iris Dataset](https://byu.instructure.com/courses/14142/files?preview=4421369)

Run K-Means on the Iris dataset using the output label as a feature and without using the output label as a feature

Requirements:
- State whether you normalize your data or not (your choice).  
- Show your results for clusterings using k = 2-7.  
- Graph the silhouette score for each k and discuss your results (i.e. what kind of clusters are being made).
---

In [ ]:
# Iris Classification without output label

In [ ]:
# Iris Classification with output label

Compare results and differences between using the output label and excluding the output label

## 2.2.2 (5%) Clustering the Iris Classification problem: K-Means

Requirements:
- Use the output label as an input feature
- Run K-Means 5 times with k=4, each time with different initial random centroids and discuss any variations in the results. 

In [ ]:
#K-Means 5 times

Discuss any variations in the results

## 3.1 (12.5%) Run the SK versions of HAC (both single and complete link) on iris including the output label and compare your results with those above.
Use the silhouette score for this iris problem(k = 2-7).  You may write your own code to do silhouette (optional extra credit) or you can use sklearn.metrics.silhouette_score. Please state if you coded your own silhouette score function to receive the extra credit points (described below). Discuss how helpful Silhouette appeared to be for selecting which clustering is best. You do not need to supply full Silhouette graphs, but you could if you wanted to.

Requirements
- Use the Sillhouette score for this iris problem (k= 2-7) 
- Use at least one other scoring function from [sklearn.metrics](https://scikit-learn.org/stable/modules/model_evaluation.html) and compare the results. State which metric was used. 
- Possible sklean metrics include (* metrics require ground truth labels):
    - adjusted_mutual_info_score*
    - adjusted_rand_score*
    - homogeneity_score*
    - completeness_score*
    - fowlkes_mallows_score*
    - calinski_harabasz_score
    - davies_bouldin_score
- Experiment using different hyper-parameters. Discuss Results

In [ ]:
# Load sklearn



*Record impressions*

## 3.2 (12.5%) Run the SK version of k-means on iris including the output label and compare your results with those above. 

Use the silhouette score for this iris problem(k = 2-7). You may write your own code to do silhouette (optional extra credit) or you can use sklearn.metrics.silhouette_score. Please state if you coded your own silhouette score function to receive the extra credit points (described below). Discuss how helpful Silhouette appeared to be for selecting which clustering is best. You do not need to supply full Silhouette graphs, but you could if you wanted to.

Requirements
- Use the Sillhouette score for this iris problem (k= 2-7) 
- Use at least one other scoring function form sklearn.metrics and compare the results. State which metric was used
- Experiment different hyper-parameters. Discuss Results

In [ ]:
# Load sklearn 



*Record impressions*

## 4. (Optional 5% extra credit) For your silhouette experiment above, write and use your own code to calculate the silhouette scores, rather than the SK or other version. 


*Show findings here*

In [ ]:
# Copy function Below